In [2]:
import xarray as xr
import rioxarray as rio
import os
from glob import glob

ImportError: cannot import name 'NDArray' from 'numpy.typing' (D:\Anaconda3\lib\site-packages\numpy\typing\__init__.py)

In [ ]:
wd='L:/cmip6_re'

os.mkdir('%s/cmip_dp' % wd)
years = list(range(2020, 2101))
for depth in ['surface', 'mesopelagic', 'bathypelagic', 'abyssopelagic']:
    os.mkdir("%s/cmip_dp/%s" % (wd, depth))
    for scen in ['ssp126', 'ssp245', 'ssp585']:
        os.mkdir("%s/cmip_dp/%s/%s" % (wd, depth, scen))
        for yr in years:
            os.mkdir("%s/cmip_dp/%s/%s/%s" % (wd, depth, scen, yr))

In [ ]:
for scen in ['ssp126', 'ssp245', 'ssp585']:
    for yr in years:
        os.chdir('%s/cmip_remap/%s/%s' % (wd, scen, yr))
        files = sorted(glob('*.nc'))
        for f in files:
            ds = xr.open_dataset(f, decode_times=False)
            dirname = f.split('_')[2]
            os.mkdir('%s/cmip_dp/surface/%s/%s/%s' % (wd, scen, yr, dirname))
            os.mkdir('%s/cmip_dp/mesopelagic/%s/%s/%s' % (wd, scen, yr, dirname))
            os.mkdir('%s/cmip_dp/bathypelagic/%s/%s/%s' % (wd, scen, yr, dirname))
            os.mkdir('%s/cmip_dp/abyssopelagic/%s/%s/%s' % (wd, scen, yr, dirname))
                
            try:
                for h,data in ds.groupby('lev'):
                    data = data.thetao 
                    h = int(round(h, 0)) 

                    # judge by depths
                    if h < 200:
                        out_name = os.path.join('%s/cmip_dp/surface/%s/%s/%s' % (wd, scen, yr, dirname), 
                                                '%s.tif' % h)
                    elif h < 1000:
                        out_name = os.path.join('%s/cmip_dp/mesopelagic/%s/%s/%s' % (wd, scen, yr, dirname), 
                                                '%s.tif' % h)
                    elif h < 4000:
                        out_name = os.path.join('%s/cmip_dp/bathypelagic/%s/%s/%s' % (wd, scen, yr, dirname), 
                                                '%s.tif' % h)
                    else:
                        out_name = os.path.join('%s/cmip_dp/abyssopelagic/%s/%s/%s' % (wd, scen, yr, dirname), 
                                                '%s.tif' % h)

                    outdata = data.transpose('time', 'lat', 'lon')
                    outdata = outdata.rio.set_spatial_dims(x_dim='lon',y_dim='lat')
                    outdata = outdata.rio.set_crs("epsg:4326")
                    outdata.rio.to_raster(out_name) 

            except:
                pass
            
        print("year: %s --- finished" % yr)
            
    print(" ## == %s --- finished == ##" % scen)